## Importing Libraries

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import re
from shapely.wkt import loads
import matplotlib.pyplot as plt
from pyproj import Proj, transform

In [ ]:
#pip install openai==0.28

In [ ]:
import os
import openai

In [ ]:
#pip install python-docx

In [ ]:
from docx import Document
from docx.shared import Inches

In [ ]:
from geopy.exc import GeopyError
import time
from ipyleaflet import Map, Marker, basemaps, WidgetControl
from ipywidgets import Button, HTML
from IPython.display import display
from ipywidgets import widgets, Layout
from geopy.geocoders import Nominatim

In [ ]:
openai.api_key ='your key'

## Opening The Data

In [ ]:
df = pd.read_csv('leeds_council_planning_apps_centroids_stx_y.csv')

In [ ]:
df['centroid_point'] = df['centroid_geom'].apply(lambda x: loads(x))

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry='centroid_point')

In [ ]:
gdf.head(5)

## Converting lat/long into Point (Area of Interest)

## Taking property location for analysis

## Only lat long selection

In [ ]:
geolocator = Nominatim(user_agent="geoapiExercises")

In [ ]:
# 定义 WGS84 坐标系（GPS 使用的坐标系）
wgs84 = Proj(init='epsg:4326')

# 定义 OSGB36 坐标系（英国国家网格使用的坐标系）
osgb36 = Proj(init='epsg:27700')

def latlon_to_eastingnorthing(lat, lon):
    # 将 WGS84 坐标转换为 OSGB36
    easting, northing = transform(wgs84, osgb36, lon, lat)
    return easting, northing


In [ ]:
def geocode(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return (location.latitude, location.longitude)
    except GeopyError as e:  # This catches any geopy-related error
        print(f"Geocoding error: {e}. Retrying...")
        time.sleep(1)  # Wait for a second and retry
        return geocode(address)
    except Exception as e:  # This catches any other unexpected error
        print(f"Unexpected error: {e}.")
    return None

In [ ]:

# Creating a map centered at an example location
m = Map(center=(53.79806386230326, -1.5437987280005416), zoom=10, basemap=basemaps.OpenStreetMap.Mapnik)

marker = Marker(name='Click me!', draggable=True)
m.add_layer(marker)

# Widget to display coordinates
coordinates = HTML()
coordinates.value = f"Click on the map to get coordinates!"

def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        marker.location = coords
        coordinates.value = f"Coordinates: {coords}"

m.on_interaction(handle_click)

widget_control = WidgetControl(widget=coordinates, position='topright')
m.add_control(widget_control)

m


In [ ]:
# lat, lon = marker.location

In [ ]:
# lat, lon

## With Address, Post Code and Map Selection

In [ ]:
# # Initialize map
# m = Map(center=(53.79806386230326, -1.5437987280005416), zoom=10, basemap=basemaps.OpenStreetMap.Mapnik)

# # Marker to show selected location, initially hidden
# marker = Marker(visible=False, draggable=False)
# m.add_layer(marker)

# # Address input widget
# address_input = widgets.Text(
#     value='',
#     placeholder='Enter address or postcode',
#     description='Address:',
#     disabled=False,
#     layout=Layout(width='50%')
# )

# # Function to update map based on address or click
# def update_map_location(coords):
#     if coords:
#         marker.location = coords
#         marker.visible = True
#         m.center = coords
#         m.zoom = 15
#     else:
#         print("Location not found.")

# # Function to handle address lookup
# def on_address_entered(b):
#     address = address_input.value
#     coords = geocode(address)
#     update_map_location(coords)

# address_input.on_submit(on_address_entered)

# # Function to handle map click
# def handle_map_click(**kwargs):
#     if kwargs.get('type') == 'click':
#         coords = kwargs.get('coordinates')
#         update_map_location(coords)

# m.on_interaction(handle_map_click)

# # Display widgets and map
# display(address_input)
# display(m)

In [ ]:
lat, lon = marker.location

In [ ]:
lat, lon

In [ ]:

wgs84 = Proj(init='epsg:4326')

osgb36 = Proj(init='epsg:27700')

def latlon_to_eastingnorthing(lat, lon):

    easting, northing = transform(wgs84, osgb36, lon, lat)
    return easting, northing


## Creating A Buffer And Extracting All Planning Applications

In [ ]:
def la_user_buffer(lat, lon):

    easting, northing = latlon_to_eastingnorthing(lat, lon)
    user_point = Point(easting, northing)
    user_buffer = user_point.buffer(250)
    points_within_user_buffer = gdf.within(user_buffer)
    data_within_user_buffer = gdf[points_within_user_buffer]
    
    return data_within_user_buffer

In [ ]:
lat, lon = 53.80127891125444, -1.560597757080992

test2=la_user_buffer(lat, lon)
test2

In [ ]:
test2.columns

In [ ]:
# import plotly.express as px

In [ ]:
# # Count the occurrence of each stage
# stage_counts = test2['stage'].value_counts().reset_index()
# stage_counts.columns = ['stage', 'count']

# # Create a pie chart with Plotly
# fig = px.pie(stage_counts, values='count', names='stage', title='Planning Application Stages Distribution')

# # Show the pie chart
# fig.show()

In [ ]:
import folium
import pandas as pd
from pyproj import Proj, transform
from folium import Circle

# Correct conversion function from Easting/Northing to Latitude/Longitude
def en_to_latlon(easting, northing):
    osgb36 = Proj(init='epsg:27700')  # British National Grid
    wgs84 = Proj(init='epsg:4326')  # WGS84
    lon, lat = transform(osgb36, wgs84, easting, northing)
    return (lat, lon)

# Apply the correct conversion to each row in the DataFrame
test2['coords'] = test2.apply(lambda x: en_to_latlon(x['easting'], x['northing']), axis=1)
test2[['latitude', 'longitude']] = pd.DataFrame(test2['coords'].tolist(), index=test2.index)

# Assuming the corrected property coordinates (from Easting/Northing conversion)
property_coords = en_to_latlon(429037.95532085805, 433975.8563029277)

# Create a map centered on the property location
m = folium.Map(location=property_coords, zoom_start=15)

# Add property marker
folium.Marker(
    property_coords,
    popup='Property Location',
    icon=folium.Icon(color='red', icon='home')
).add_to(m)

# Filter for 'DETAILED PLANS SUBMITTED' and 'DETAILED PLANS APPROVED'
applications_to_plot = test2[test2['stage'].isin(['DETAILED PLANS SUBMITTED', 'DETAILED PLANS APPROVED'])]

# Add markers for planning applications
for _, row in applications_to_plot.iterrows():
    folium.Marker(
        location=(row['latitude'], row['longitude']),
        popup=f"{row['stage']} Planning Application",
        icon=folium.Icon(color='green' if row['stage'] == 'DETAILED PLANS APPROVED' else 'orange')
    ).add_to(m)
    
# Draw a 250-meter buffer around the property location
Circle(
    location=property_coords,
    radius=250,  # radius in meters
    color='blue',
    fill=True,
    fill_color='blue',
    fill_opacity=0.1,
    popup='250m Buffer'
).add_to(m)

# Display the map
m.save('map.html')
m

In [ ]:
from pyproj import Proj, transform

# Latitude and Longitude in WGS84
latitude = 53.80127891125444
longitude = -1.560597757080992

# Define the WGS84 projection (used for GPS coordinates)
wgs84 = Proj(init='epsg:4326')

# Define the British National Grid projection
bng = Proj(init='epsg:27700')

# Perform the conversion from WGS84 to British National Grid
easting, northing = transform(wgs84, bng, longitude, latitude)

print(f"Easting: {easting}, Northing: {northing}")


In [ ]:
#app_pp.to_csv('test2.csv')

## Visualization

In [ ]:
import folium
import pandas as pd
from pyproj import Transformer

# 假设 test2 已经定义，并包含 'easting' 和 'northing' 列

# 创建一个坐标转换器（OSGB36 to WGS84）
transformer = Transformer.from_crs("epsg:27700", "epsg:4326", always_xy=True)

# 将 'easting' 和 'northing' 坐标转换为纬度和经度
test2[['longitude', 'latitude']] = test2.apply(
    lambda row: transformer.transform(row['easting'], row['northing']), axis=1, result_type='expand'
)

# 创建一个新的地图，以转换后的坐标为中心
map_center = [test2['latitude'].mean(), test2['longitude'].mean()]
m_ = folium.Map(location=map_center, zoom_start=16)

# 添加转换后坐标的标记
for idx, row in test2.iterrows():
    folium.Marker([row['latitude'], row['longitude']], popup=row['proposal']).add_to(m_)

# 使用提供的纬度和经度作为中心点
lat, lon = 53.80127891125444, -1.560597757080992

# 在地图上添加红色的中心点
folium.Marker(
    [lat, lon],
    icon=folium.Icon(color='red'),
    popup="Target Location"
).add_to(m_)

# 绘制一个半径为250米的红色圆圈作为范围
folium.Circle(
    radius=250,
    location=[lat, lon],
    color='red',
    fill=True,
    fill_color='red',
    popup="250m"
).add_to(m_)

m_.save('porposal_location.html')
m_


In [ ]:
# api_key = 'your key'

## Other analysis

### Locate the coordinates at LSOA

In [ ]:
from shapely.geometry import Point, Polygon
import json

# Latitude and longitude provided by the user

point = Point(lon, lat)

# Load the TopoJSON file to get the geometries
with open("topo_E08000035.json") as file:
    topo_data = json.load(file)

# Function to convert TopoJSON arc indices to coordinate pairs
def topojson_to_coordinates(arc_indices, arcs):
    coordinates = []
    for index in arc_indices:
        if index >= 0:
            coordinates += arcs[index]
        else:
            coordinates += arcs[~index][::-1]
    return coordinates

# Iterate over the geometries in the TopoJSON file to find the LSOA
lsoa_code = None
lsoa_name = None
for feature in topo_data["objects"]["E08000035"]["geometries"]:
    if feature["type"] == "Polygon":
        polygon_coordinates = topojson_to_coordinates(feature["arcs"][0], topo_data["arcs"])
        polygon = Polygon(polygon_coordinates)
        if point.within(polygon):
            lsoa_code = feature["properties"]["LSOA11CD"]
            lsoa_name = feature["properties"]["LSOA11NM"]
            break
    elif feature["type"] == "MultiPolygon":
        for polygon_arc in feature["arcs"]:
            polygon_coordinates = topojson_to_coordinates(polygon_arc[0], topo_data["arcs"])
            polygon = Polygon(polygon_coordinates)
            if point.within(polygon):
                lsoa_code = feature["properties"]["LSOA11CD"]
                lsoa_name = feature["properties"]["LSOA11NM"]
                break
    if lsoa_code:
        break

lsoa_code, lsoa_name


## Use LSOA to generate crime , air pollution and medical report

In [ ]:

# Load the dataset
file_path = 'physical_activity_v6.csv'
data = pd.read_csv(file_path)

In [ ]:
def generate_report(lsoa_code):
    # Find the row for the specified LSOA
    lsoa_data = data[data['LSOA21CD'] == lsoa_code].iloc[0]
    
    # Extract required data
    crime_rate = lsoa_data['%Crime']
    air_quality = lsoa_data['air_quality']
    health_index = lsoa_data['%Health_Cov']
    
    # Initialize report components
    crime_report = ""
    air_quality_report = ""
    health_index_report = ""
    
    # Generate crime rate report
    crime_messages = [
        {"role": "system", "content": "You are a professional real estate analyst."},
        {"role": "user", "content": f"Explain the significance of a crime rate of {crime_rate}% in an LSOA area.The average crime rate is 11.7. give short summery."}
    ]
    crime_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=crime_messages
    )
    crime_report = "Crime Rate Report: " + crime_response['choices'][0]['message']['content']
    
    # Generate air quality report
    air_quality_messages = [
        {"role": "system", "content": "You are a professional real estate analyst."},
        {"role": "user", "content": f"Analyze the air quality index of {air_quality} in an LSOA area.give short summery."}
    ]
    air_quality_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=air_quality_messages
    )
    air_quality_report = "Air Quality Report: " + air_quality_response['choices'][0]['message']['content']
    
    # Generate health index report
    health_index_messages = [
        {"role": "system", "content": "You are a professional real estate analyst."},
        {"role": "user", "content": f"Discuss the importance of a health index of {health_index} in an LSOA area, indicating the availability of medical resources.give short summery."}
    ]
    health_index_response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=health_index_messages
    )
    health_index_report = "Health Index Report: " + health_index_response['choices'][0]['message']['content']
    
    # Return reports as a tuple
    return (crime_report, air_quality_report, health_index_report)

In [ ]:
crime_report, air_quality_report, health_index_report = generate_report(lsoa_code)

In [ ]:
crime_report

In [ ]:
air_quality_report

In [ ]:
health_index_report

## House price analysis

In [ ]:
from house_price import House_price

my_instance = House_price(lat, lon)

area_average_price, annual_average_prices = my_instance.average_price()

In [ ]:
area_average_price, annual_average_prices

In [ ]:
my_instance.line_map()


In [ ]:
m = my_instance.folium_map()
m.save('house_price_location.html')
m

## Ranking for Planning Applications

In [ ]:
def get_scores_from_api(user_role,proposals):


    messages = [
        {"role": "system", "content": f"As a professional real estate analyst"},
        {"role": "user", "content": f"proposals: {proposals}.please rate the following future planning proposals related to the real estate near 250m on a scale of 0 to 10. This is the score of the proposal's impact on all aspects of the {user_role}'s purchase of the property. The higher the score, the greater the impact of the proposal on the future of the property.Do not delete any proposals. The answer format is: 1.score=x, Text of the proposal. 2.score=x, Text of the proposal. 3.score=x, Text of the proposal....."}
    ]
        
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        max_tokens=4096
    )

    
    return response



In [ ]:
app_pp = test2[test2['stage'].isin(['DETAILED PLANS SUBMITTED','DETAILED PLANS APPROVED'])]

In [ ]:
proposals_text = app_pp['proposal'].str.cat(sep='\n\n')

In [ ]:
user_role = 'home buyer'
rated_proposals= get_scores_from_api(user_role,proposals_text)

In [ ]:
rated_proposals

In [ ]:
rated_text = rated_proposals.choices[0].message.content

pattern = r'(\d+)\.\s*score=(\d+),\s*(.*?)\s*(?=\d+\.\s*score=|$)'


matches = re.findall(pattern, rated_text, re.DOTALL)

df_auto = pd.DataFrame(matches, columns=['index', 'score', 'proposal'])


df_auto['index'] = df_auto['index'].astype(int)
df_auto['score'] = df_auto['score'].astype(int)

df_auto  # Display the corrected DataFrame to verify



In [ ]:
filtered_proposal = df_auto[df_auto['score'] > 5]

filtered_proposal

## Comparative Analysis for all planning applications

In [ ]:
import openai
import pandas as pd

def analyze_planning_applications(user_role, data):
    """
    根据用户角色和规划应用程序数据集全面分析每个规划提案，生成房产规划建议报告。
    
    参数:
    - user_role: 用户角色（'buyer'，'seller'，'agent'）
    - dataset_path: 规划应用程序数据集的路径
    """

    
    # 分类应用
    submit = data[data['stage'] == 'DETAILED PLANS SUBMITTED']
    approved = data[data['stage'] == 'DETAILED PLANS APPROVED']
    withdrawn = data[data['stage'] == 'WITHDRAWN']
    refused = data[data['stage'] == 'REFUSED']
    
    # 构建 ChatGPT 请求的上下文，包括每个分类的所有提案
    context = f"""
    As a professional property analyst, I am analysing planning application proposals within 250m of specific properties for {user_role}. This dataset contains planning applications approved, submitted, withdrawn and rejected. The structure of the generated article is as follows: The four types of planning applications are analyzed one by one: approval, submission, withdrawal and rejection, and each type of planning application is compared and analyzed. Next, compare and analyze the reasons why some proposals were withdrawn or rejected. Finally, please compare the planning permission that has been granted for the property with the planning proposals that have been withdrawn and rejected, format the answers in html, Headings use h4, h5, h6level.
    Submit applications details: {submit[['reference', 'proposal']].to_dict(orient='records')}
    Approved applications details: {approved[['reference', 'proposal']].to_dict(orient='records')}
    Withdrawn applications details: {withdrawn[['reference', 'proposal']].to_dict(orient='records')}
    Refused applications details: {refused[['reference', 'proposal']].to_dict(orient='records')}
    """
    # 调用 ChatGPT API 生成报告
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a real estate analyst providing comprehensive analysis."},
            {"role": "user", "content": context}
        ],
    )
    
    # 打印生成的报告
    report = response['choices'][0]['message']['content']
    print(report)
    return report.strip()


In [ ]:
comparative_Analysis = analyze_planning_applications(user_role, test2)

## Benefits and Harms, Risks and Opportunities for approved and submitted applications

In [ ]:
def generate_report_for_role(user_role, planning_applications):
    """
    Generates a Target Audience Report tailored to the user role using model "gpt-3.5-turbo" and the chat completions endpoint.
    
    Parameters:
    - user_role: str, the role of the user.
    - planning_applications_insights: str, insights from planning applications.
    
    Returns:
    - str, customized Target Audience Report.
    """
    # Setup the messages based on user role
    messages = [{
        'role': 'system',
        'content': f"As a professional real estate analyst, Generate a dominance Report based on the following planning applications insights. What are benefits and harms, risks and opportunities for a {user_role}? There is no need to give a summary at the end,format the answers in html, Headings use h4, h5, h6level"
    }, {
        'role': 'user',
        'content': f"{planning_applications}"
    }]
    
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    # Assuming the last message contains the report
    report = response['choices'][0]['message']['content']
    return report.strip()



In [ ]:
# Generate reports for different roles
proposals = filtered_proposal['proposal']
dominance_report = generate_report_for_role(user_role, proposals)

print(dominance_report)


## Detailed analysis for most impactful applications

In [ ]:
impactful_proposal = df_auto[df_auto['score'] >= 8]

impactful_proposal

In [ ]:
example = """
<h4>the content of the Proposal</h4>
<h5>Impact:</h5>
<ul>
  <li>content of the impact</li>
</ul>

<h5>Risks:</h5>
<ul>
  <li>content of the risk.</li>
</ul>
"""

In [ ]:
def analyze_proposal(proposal_description, user_role, example):

    messages = [
        {
            "role": "system",
            "content": f"A professional real estate analyst is analyzing planning application proposals within 250m of a specific property for a {user_role}. Provide detailed insights into the proposals to help understand the most important developments and the risks associated with purchasing the property with highlighting the planning application and write its impact in bullet format.Don't add a conclusion at the end, format the answers in html,the example format is{example}" 

        },
        {
            "role": "user",
            "content": f"Description: {proposal_description}.Don't add a conclusion at the end"
        }
    ]
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.5,
            max_tokens=4000
        )
        
        # 提取并打印分析结果
        print(f"Analysis for Proposal Reference: {proposal_description}")
        # 注意：根据 API 返回的具体结构可能需要调整
        for message in response['choices']:
            print(message['message']['content'])

    except Exception as e:
        print(f"Error calling OpenAI API for proposal reference {proposal_description}: {e}")
        
    report = response['choices'][0]['message']['content']
    return report.strip()

In [ ]:
detaileds = []
for index, proposal in impactful_proposal.iterrows():
    detailed = analyze_proposal(proposal.get('proposal', 'N/A'), user_role, example)
    detaileds.append(detailed)

## Environmental Impact

In [ ]:
def analyze_martello(mar_data, user_role):

    messages = [
        {
            "role": "system",
            "content": f"Professional real estate analysts are analyzing data on specific properties for {user_role}. Provide detailed insight into the proposal to help understand the environmental impact on property and the risks associated with purchasing the property. Give very short description for each factor with title. Don't add a conclusion to the end, format the answers in html,Headings use h4, h5, h6level, Do not include line breaks in the answer\n"
        },
        {
            "role": "user",
            "content": f"Description: {mar_data}.Don't add a conclusion at the end"
        }
    ]
    
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=4096,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")

    report = response['choices'][0]['message']['content']
    print(report)
    
    return report.strip()

In [ ]:
mar_info="Active Landfill (+): The title is more than 1km from any active landfill. COMAH Establishments (-): The Health and Safety Executive (HSE) indicates that the title is likely to be within a Control of Major Accident Hazard (COMAH) site Public Information Zone (PIZ) - the threshold being 10km for Upper Tier COMAH sites and 1km for Lower Tier COMAH sites. COMAH applies to the industries where threshold quantities of dangerous substances are identified. COMAH sites are highly regulated to minimise risks to the environment and people. Proximity to a COMAH site may cause risks in the event of an accident, and it could potentially impact future development of the title. To obtain specific and up-to-date information about site PIZs, you should contact the relevant Local Authority, Emergency Services, or the operator of the COMAH site in question. Pluvial (+):No flood risk issues of concern were identified. Flood hazards are either not present in the search area or are considered very low likelihood and severity. Historical Flood Areas (-): The property is not in an area which has been flooded in the past, however it is within 250 meters of a previous flood. Green Belt (+): The title is not within a green belt designated area. Nature Reserves (+): The title is more than 250 metres of a Local or National Nature Reserve. Listed Buildings (-): The title is within 250 metres of 1 or more Listed Buildings. Owning or living near a listed building may require adherence to strict regulations for alterations or renovations to preserve its historic or architectural value. Refer to section Certificates of Immunity to check whether immunity has been granted against certain planning restrictions. Radon (-): The title is in a Radon Affected Area where 3% to 5% of homes are estimated to be at or above the Action Level. Overground Rail (+): The title is more than 250 metres away from a railway station or line. Landslides (+): No indicators for slope instability identified. No special actions required to avoid problems due to landslides. No special ground investigation required, and increased construction costs or increased financial risks are unlikely due to potential problems with landslides."

In [ ]:
martello=analyze_martello(mar_info, user_role)

## Conclusion

In [ ]:
import openai



def generate_planning_proposal_summary(user_role, comparative_Analysis, dominance_report, detaileds, martello, crime_report, air_quality_report, health_index_report):
    """
    根据用户角色和规划提案的具体分析生成总结报告。

    参数:
    user_role: 用户角色（'buyer', 'seller', 'agent'）
    proposal_analyses: 规划提案的具体分析列表
    """
    # 构建输入消息
    messages = [
        {"role": "system", "content": f"As a professional real estate analyst, Generate a Conclusion  based on the following planning applications insights for a {user_role}.Write the summary in a 250 word paragraph，just one paragraph,format the answers in html, Headings use h4, h5, h6level, Do not include line breaks in the answer\n"},
        {"role": "user", "content": f" Below is planning application proposals within 250m of a specific property.{detaileds},The report contains the following contents, Comparative Analysis:{comparative_Analysis}, dominance report: {dominance_report} ,martello{martello},{crime_report}, {air_quality_report}, {health_index_report}"}
    ]
    

    # 调用 ChatGPT API
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=4096,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")

    report = response['choices'][0]['message']['content']
    print(report)
    
    return report.strip()

In [ ]:
summary = generate_planning_proposal_summary(user_role, comparative_Analysis, dominance_report, detaileds, martello, crime_report, air_quality_report, health_index_report)

## Recommendations

In [ ]:
import openai



def generate_recommendation(user_role, comparative_Analysis, dominance_report, detaileds, martello, crime_report, air_quality_report, health_index_report):
    """
    根据用户角色和规划提案的具体分析生成总结报告。

    参数:
    user_role: 用户角色（'buyer', 'seller', 'agent'）
    proposal_analyses: 规划提案的具体分析列表
    """
    # 构建输入消息
    messages = [
        {"role": "system", "content": f"As a professional real estate analyst, Generate a general Recommendations based on the following property report,Cross-analyze to make the recommendations, To help {user_role} make informed choices about property transaction.format the answers in html, Headings use  h5, h6level,Do not include line breaks in the answer\n"},
        {"role": "user", "content": f" Below is planning application proposals within 250m of a specific property.{detaileds},The report contains the following contents, Comparative Analysis:{comparative_Analysis}, dominance report: {dominance_report} ,martello{martello},{crime_report}, {air_quality_report}, {health_index_report}"}
    ]
    

    # 调用 ChatGPT API
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=4096,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")

    report = response['choices'][0]['message']['content']
    print(report)
    
    return report.strip()

In [ ]:
Recommendation = generate_recommendation(user_role, comparative_Analysis, dominance_report, detaileds, martello, crime_report, air_quality_report, health_index_report)

## Decision

In [ ]:
import openai



def generate_decision(user_role, comparative_Analysis, dominance_report, detaileds, martello, crime_report, air_quality_report, health_index_report, Recommendation):
    """
    根据用户角色和规划提案的具体分析生成总结报告。

    参数:
    user_role: 用户角色（'buyer', 'seller', 'agent'）
    proposal_analyses: 规划提案的具体分析列表
    """
    # 构建输入消息
    messages = [
        {"role": "system", "content": f" As a professional real estate analyst, giving a overall insightful overview that can help {user_role} to take decision of this property transation. give the decide of whether or not to buy this property. 100 word limit, one paragraph.format the answers in html, Headings use h4, h5, h6level,Do not include line breaks in the answer\n"},
        {"role": "user", "content": f" here is extensive analysis on a property,Below is planning application proposals within 250m of a specific property.{detaileds},The report contains the following contents, Comparative Analysis:{comparative_Analysis}, dominance report: {dominance_report} ,martello{martello},{crime_report}, {air_quality_report}, {health_index_report},Recommendation:{Recommendation}"}
    ]
    

    # 调用 ChatGPT API
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=4096,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")

    report = response['choices'][0]['message']['content']
    print(report)
    
    return report.strip()

In [ ]:
decision = generate_decision(user_role, comparative_Analysis, dominance_report, detaileds, martello, crime_report, air_quality_report, health_index_report,Recommendation)

## Brief

In [ ]:
def generate_brief(user_role, basic_info):

    # 构建输入消息
    messages = [
        {"role": "system", "content": f"As a professional real estate analyst.format the answers in html, Headings use h4, h5, h6level,Do not include line breaks in the answer\n"},
        {"role": "user", "content": f"Please write a basic description of the house accformat the answers in htmlording to the information given, basic informauser_roletion: {basic_info}"}
    ]
    

    # 调用 ChatGPT API
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=4096,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")

    report = response['choices'][0]['message']['content']
    print(report)
    
    return report.strip()

In [ ]:
basic_info="House name: 43 Hanover Square,FLAT 3 43 Hanover Square in Leeds is a leasehold 1-bedroom mid-terrace flat with an EPC rating of D. This property has the UPRN (Unique Property Reference Number) 72533861 and was last sold for £238500 on 15/09/2022. Total Floor Area 39.0 sq. metres, Floor Height 2.2 metres, Habitable Rooms 2 rooms, Bedroom 1, Reception room 1, Built form Mid-Terrace."

In [ ]:
brief=generate_brief(user_role, basic_info)

## Introduction 

In [ ]:
def generate_introduction(user_role, basic_info, repo_info):

    # 构建输入消息
    messages = [
        {"role": "system", "content": f"As a professional real estate analyst.format the answers in html, Headings use h4, h5, h6level,Do not include line breaks in the answer\n"},
        {"role": "user", "content": f"please summarize the introduction of the report for {user_role}. report_information:{repo_info} "}
    ]
    

    # 调用 ChatGPT API
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.7,
            max_tokens=4096,
            top_p=1.0,
            frequency_penalty=0.0,
            presence_penalty=0.0
        )
        

    except Exception as e:
        print(f"Error calling OpenAI API: {e}")

    report = response['choices'][0]['message']['content']
    print(report)
    
    return report.strip()

In [ ]:
repo_info = "This report provides an in-depth analysis of the property, offering insights into local planning applications, environmental factors, amenities, public transportation, crime rates, and air quality. These elements are crucial for making an informed decision about your potential new home."

In [ ]:
introduction = generate_introduction(user_role, basic_info, repo_info)

## Final Report

In [ ]:
doc = Document()

doc_title = "Report for {}".format(user_role)
doc.add_heading(doc_title, level=1)

doc.add_heading('Introduction', level=2)
doc.add_paragraph(introduction)

doc.add_heading('Property Report Overview ', level=3)
doc.add_paragraph(brief)

image_path1 = 'Map with Planning Application .png'  # Replace with the path to your image
doc.add_picture(image_path1, width=Inches(4), height=Inches(3.0))
doc.add_paragraph("Map with Planning Application")

doc.add_heading('House Price Trend', level=3)
image_path2 = 'house_price.png'
doc.add_picture(image_path1, width=Inches(4), height=Inches(3.0))
doc.add_paragraph("House Price")

doc.add_heading('Analysis', level=1)

doc.add_heading('Comparative Analysis for all the appplications', level=2)
doc.add_paragraph(comparative_Analysis)

doc.add_heading('Dominance Report for approved and submitted appplications', level=2)
doc.add_paragraph(dominance_report)

detaileds_content = '\n'.join(detaileds)
doc.add_heading('Detailed analysis for most impactful applications', level=2)
doc.add_paragraph(detaileds_content)

doc.add_heading('Environmental Report', level=2)
doc.add_paragraph(martello)

doc.add_heading('Other Data Report', level=2)
doc.add_heading('Crime Report', level=3)
doc.add_paragraph(crime_report)
doc.add_heading('air pollution Report', level=3)
doc.add_paragraph(air_quality_report)
doc.add_heading('Medical index Report', level=3)
doc.add_paragraph(health_index_report)

doc.add_heading('Dominance report', level=2)
doc.add_paragraph(dominance_report)

doc.add_heading('Recommendations', level=2)
doc.add_paragraph(Recommendation)

doc.add_heading('Conclusion', level=2)
doc.add_paragraph(summary)

doc_name = "report for {}.docx".format(user_role)

doc.save(doc_name)


In [ ]:
doc_name

In [ ]:


detaileds = [item.replace('\n', '') for item in detaileds]
image_path1 = 'Map with Planning Application .png' 
image_path2 = 'house_price.png'

# Define the HTML content
html_content = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Report for {user_role}</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 20px; }}
        h1, h2, h3 {{ color: navy; }}
        p {{ text-align: justify; }}
        img {{ width: 400px; height: auto; display: block; margin: 20px 0; }}
    </style>
</head>
<body>
    <h1>Report for {user_role}</h1>

    <h2>Introduction</h2>
    <p>{introduction}</p>

    <h3>Property Report Overview</h3>
    <p>{brief}</p>
    
    <iframe src="porposal_location.html" style="width: 60%; height: 500px; border: none;"></iframe>
    <p>Map with Planning Application</p>

    <h3>House Price Trend</h3>
    <img src="{image_path2}" alt="House Price Trend">
    <p>House Price Trend</p>
    <h3>House Price Location</h3>
    <iframe src="house_price_location.html" style="width: 60%; height: 500px; border: none;"></iframe>
    <p>Map with House Price Location</p>

    <h2>Analysis</h2>
    <h3>Comparative Analysis for all the applications</h3>
    <p>{comparative_Analysis}</p>

    <h3>Dominance Report for approved and submitted applications</h3>
    <p>{dominance_report}</p>

    <h3>Detailed analysis for most impactful applications</h3>
    <p>{detaileds}</p>

    <h2>Environmental Report</h2>
    <p>{martello}</p>

    <h2>Other Data Report</h2>
    <h3>Crime Report</h3>
    <p>{crime_report}</p>

    <h3>Air Pollution Report</h3>
    <p>{air_quality_report}</p>

    <h3>Medical Index Report</h3>
    <p>{health_index_report}</p>

    <h2>Recommendations</h2>
    <p>{Recommendation}</p>

    <h2>Conclusion</h2>
    <p>{summary}</p>
</body>
</html>
"""

# Specify the path and save the HTML content to a file with UTF-8 encoding
html_file_path = "report_for_{}.html".format(user_role.lower().replace(" ", "_"))
with open(html_file_path, 'w', encoding='utf-8') as file:
    file.write(html_content)

# Provide the path for user to download or view the generated HTML file
html_file_path
